In [1]:
import pandas as pd
import numpy as np
from os import listdir

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import random
import math

In [2]:
def HeaderMaker(PatientFile):
    df_cleaned = pd.read_csv('/data/ortho/' + PatientFile, header=None)

    df_cleaned = df_cleaned.rename(columns={0: "thorax_r_x", 1: "thorax_r_y", 2: "thorax_r_z"})
    df_cleaned = df_cleaned.rename(columns={3: "clavicula_r_x", 4: "clavicula_r_y", 5: "clavicula_r_z"})
    df_cleaned = df_cleaned.rename(columns={6: "scapula_r_x", 7: "scapula_r_y", 8: "scapula_r_z"})
    df_cleaned = df_cleaned.rename(columns={9: "humerus_r_x", 10: "humerus_r_y", 11: "humerus_r_z"})
    df_cleaned = df_cleaned.rename(columns={12: "ellebooghoek_r"})
    df_cleaned = df_cleaned.rename(columns={15: "thorax_l_x", 16: "thorax_l_y", 17: "thorax_l_z"})
    df_cleaned = df_cleaned.rename(columns={18: "clavicula_l_x", 19: "clavicula_l_y", 20: "clavicula_l_z"})
    df_cleaned = df_cleaned.rename(columns={21: "scapula_l_x", 22: "scapula_l_y", 23: "scapula_l_z"})
    df_cleaned = df_cleaned.rename(columns={24: "humerus_l_x", 25: "humerus_l_y", 26: "humerus_l_z"})
    df_cleaned = df_cleaned.rename(columns={27: "ellebooghoek_l"})
    
    return df_cleaned

In [3]:

Patients = {}
testdataurl = '/data/ortho/'
for PatientFile in listdir(testdataurl):
    if ('meting' not in PatientFile):
        print('Skipping: %s' % PatientFile)
        # Skip this iteration
        continue
    Splitted = PatientFile.split('_')
    Patient = Splitted[0] + '_' + Splitted[1]

    Data = HeaderMaker(PatientFile)
    MaxDict = Data.max().to_dict() # .abs() for absolute numbers
    
    if(Patient not in Patients.keys()):
        Patients[Patient] = MaxDict
        Patients[Patient]['GroundTruth'] = Splitted[0].split('Cat')[1]
        Patients[Patient]['Bias'] = 0
        continue
    
    for key in MaxDict.keys():
        Patients[Patient][key]= max(Patients[Patient][key],MaxDict[key])
        

    
    
            
            
            
            

Skipping: StudentNummers
Skipping: testset
Skipping: AllPatients.csv
Skipping: TestDataAllPatients.csv


In [4]:
df = pd.DataFrame.from_dict(Patients, orient='index')
# print(df.columns.values)
# df = df[['Cat2' in index or 'Cat3' in index for index in df.index]] # Welke colommen wil je houden

# df['TrueFalse'] = ['2' == i for i in df['GroundTruth']]

# print(df.head())

In [5]:
Xcolumns = ['Bias']
Xcolumns.extend(['thorax_r_x', 'thorax_r_y', 'thorax_r_z', 'clavicula_r_x', 'clavicula_r_y',
 'clavicula_r_z', 'scapula_r_x', 'scapula_r_y', 'scapula_r_z', 'humerus_r_x',
 'humerus_r_y', 'humerus_r_z', 'ellebooghoek_r', 'thorax_l_x',
 'thorax_l_y', 'thorax_l_z', 'clavicula_l_x', 'clavicula_l_y', 'clavicula_l_z',
 'scapula_l_x', 'scapula_l_y', 'scapula_l_z', 'humerus_l_x', 'humerus_l_y',
 'humerus_l_z', 'ellebooghoek_l'])

# X = df[Xcolumns]
# y = df[['GroundTruth']]




In [6]:
print(df['GroundTruth'].unique())

['1' '2' '3' '4']


In [7]:
def Patient_test_split(dataframe, xcol, ycol, Percentage):
    allIndex = np.unique(df.index.tolist()) # Get all unique patients id's
    random.seed(2) # Set random seeed so the answer is the same
    
    PercentageIndex = []
    for i in df['GroundTruth'].unique(): # Cycle through each categorie (only cycles through the ones that are present)
        CatPatients = np.array([i.split('_')[0].split('Cat')[1] for i in allIndex])
        CatPatients = CatPatients[CatPatients==i]
        
        AmountItems = len(CatPatients)
        AmountRandom = math.floor(AmountItems*Percentage)
        PercentageIndex.extend(random.sample(list(CatPatients), AmountRandom))        
        
    AmountItems = len(allIndex)
    AmountRandom = math.floor(AmountItems*Percentage)
    
    PercentageIndex = random.sample(list(allIndex), AmountRandom)
    
    Percentagedf = df[xcol][df.index.isin(PercentageIndex)]
    Percentagey = df[ycol][df.index.isin(PercentageIndex)]
    
    Testdf = df[xcol][~df.index.isin(PercentageIndex)]
    Testy = df[ycol][~df.index.isin(PercentageIndex)]
    print('Final Percentage test: %s' % (len(Testdf)/len(Percentagedf)))
    return (Percentagedf, Testdf, Percentagey, Testy)

X_train, X_test, y_train, y_test = Patient_test_split(df, Xcolumns, 'GroundTruth', 0.85)

Final Percentage test: 0.1797752808988764


In [8]:
print(len(X_train), len(X_test))

89 16


In [9]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [10]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [11]:
y_pred = lr.predict(X_test)
print(lr.coef_[0])
print(metrics.accuracy_score(y_test, y_pred))
print(y_test)

[ 0.00000000e+00 -2.68809879e-01 -1.83328002e-01 -1.65797670e-01
  2.89572322e-02  2.82719682e-01  1.41958743e-01  5.57515112e-02
 -1.04402045e-01 -2.01384418e-01 -1.52717496e-02  2.56102077e-01
 -3.54379245e-02 -1.82435968e-04 -2.68809879e-01  2.55939653e-02
 -2.18431373e-01 -1.58411910e-01 -3.18493185e-01 -1.76325480e-03
 -1.22925231e-01 -3.21649224e-01 -7.58086366e-03 -1.61328203e-01
 -8.21607908e-02  6.07288012e-02  8.70534215e-05]
0.5
Cat1_pat12    1
Cat1_pat3     1
Cat1_pat7     1
Cat2_pat1     2
Cat2_pat10    2
Cat2_pat12    2
Cat2_pat20    2
Cat2_pat26    2
Cat2_pat31    2
Cat2_pat34    2
Cat2_pat6     2
Cat3_pat15    3
Cat3_pat20    3
Cat3_pat25    3
Cat3_pat26    3
Cat3_pat37    3
Name: GroundTruth, dtype: object


In [15]:
from sklearn.metrics import mean_squared_error, \
recall_score, precision_score, accuracy_score, confusion_matrix

y_true = y_test
y_pred =lr.predict(X_test)

print(metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           1       0.67      0.67      0.67         3
           2       1.00      0.12      0.22         8
           3       0.50      1.00      0.67         5
           4       0.00      0.00      0.00         0

   micro avg       0.50      0.50      0.50        16
   macro avg       0.54      0.45      0.39        16
weighted avg       0.78      0.50      0.44        16



/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [16]:
print(df)

            thorax_r_x  thorax_r_y  thorax_r_z  clavicula_r_x  clavicula_r_y  \
Cat1_pat1    13.663678    2.179559    1.732670      12.444494      59.705033   
Cat1_pat10   -0.989045    1.973656   -1.828659      -0.041210      44.600980   
Cat1_pat11    4.897893    4.136994    3.735479      -8.096654      33.784876   
Cat1_pat12    0.746364    1.606608    0.696937     -25.972016      20.651558   
Cat1_pat13    2.312612    4.934213   -0.315249     -10.660820      37.444309   
Cat1_pat14    4.162713    5.058497    4.803960      -7.861483      34.299972   
Cat1_pat15    2.286132    4.285242    5.074372     -18.481031      43.643171   
Cat1_pat16    0.356370    2.071765    2.271676      -7.684299      33.370916   
Cat1_pat17    8.925056    6.575340    1.149593     -16.410715      32.036554   
Cat1_pat18    0.134793   -0.103062   -0.050370     -10.625133      49.649160   
Cat1_pat19    1.394970    4.273942    3.573050      -9.331682      37.832953   
Cat1_pat2    -4.619013   -1.060508    6.